In [ ]:
! pip install xlrd

In [2]:
import pandas_profiling as pp
import pandas as pd

In [3]:
! ls -l ../data/

total 12
drwxrwxr-x 3 ubuntu ubuntu 4096 Jan 25 20:30 original
drwxrwxr-x 3 ubuntu ubuntu 4096 Jan 25 20:28 report
drwxrwxr-x 3 ubuntu ubuntu 4096 Jan 25 20:36 resources


In [4]:
! head ../data/DB_V2.TXT

head: cannot open '../data/DB_V2.TXT' for reading: No such file or directory


In [5]:
df = pd.read_csv("../data/original/DB_V2.TXT", sep='\t', encoding='latin-1')

/home/ubuntu/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.drop('Year', axis=1, inplace=True)
df.drop('account_yy_ss', axis=1, inplace=True)

In [9]:
df.sample(5)

,Province,Type,hosp_serv_id,reimbt_cat_id,drug_code,realization_date,quantity,amount_reimb,amount_not_reimb
9318191,Flandre orientale,Général,990,750912,740605,20071,2,328.08,0.00
15174604,Brabant Flamand,Général,210,757245,779686,20111,15,0.00,2394.18
11542691,Liège,Général,210,756626,7799976,20074,-9,0.00,-3.95
11378549,Limbourg,Psychiatrique,370,750820,113597,20082,102,0.00,24.19
7211477,Brabant Wallon,Général,210,753745,762492,20061,24,10.11,2.22


In [36]:
df

,Province,Type,hosp_serv_id,reimbt_cat_id,drug_code,realization_date,quantity,amount_reimb,amount_not_reimb
0,Anvers,Général,990,698014,0,20014,1,218.10,0.00
1,Anvers,Général,990,698051,0,20014,3,5816.01,0.00
2,Anvers,Général,990,698051,0,20014,1,51.83,0.00
3,Anvers,Général,990,698051,0,20021,2,535.21,0.00
4,Anvers,Général,990,698051,0,20021,1,267.61,0.00
5,Anvers,Général,720,698051,0,20022,1,5184.17,0.00
6,Anvers,Général,720,698051,0,20022,1,5184.17,0.00
7,Anvers,Général,990,698051,0,20022,7,6789.83,0.00
8,Anvers,Général,990,698051,0,20023,4,1070.44,0.00
9,Anvers,Général,220,698062,0,20012,1,676.70,0.00


    100% |████████████████████████████████| 112kB 3.1MB/s ta 0:00:01
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
pharma_ref = pd.read_excel("../data/original/PHARMA_REF.xlsx")

In [24]:
pharma_ref.sample(5).T

,23009,1182,27165,14006,8340
trim_pharma,20183,20183,20183,20183,20183
drug_code,2760478,4000782,1658376,736959,284185
produit_pharma,2760478,4000782,1658376,736959,284185
type_drug_code,Package,Package,Package,Unit,NaN
famille_produit_pharma,Spécialité pharmaceutique,Spécialité pharmaceutique,Médicament D,Spécialité pharmaceutique,NaN
drug_name_aggregated,isotretinoine - generieken,OXYGENE gazeux,MENINGITEC,HUMULINE,NaN
nom_produit_pharma,ISOTRETINOINE MYLAN 20 mg,OXYGENE MEDICAL GAZEUX IJSFABRIEK STROMBEEK,"MENINGITEC FL 1 X 0,5ML + KIT INJ",HUMULINE NPH,NaN
conditionnement,100 capsules molles x 20 mg Isotrétinoïne en ...,"cyl. 5 l - 0,75 m?",voir déno,* pr. 1 ml x 100 I.U./ml,NaN
mode_administration,COMPRIMES,INHALATION / NASAL,INJECTION,INJECTION,NaN
nom_firme_pharma,MYLAN,IJSFABRIEK STROMBEEK,NaN,ELI LILLY BENELUX,NaN


In [33]:
pharma_ref["drug_code"] = pharma_ref["drug_code"].astype("category")

In [14]:
reimburse_cat = pd.read_excel("../data/original/reimb category.xlsx")

In [25]:
reimburse_cat.sample(5).T

,3,658,441,515,690
reimbt_cat_id,698025,759382,746233,747036,759673
dbegin,1988-01-01 00:00:00,2017-01-01 00:00:00,2015-06-01 00:00:00,2015-06-01 00:00:00,2018-01-01 00:00:00
dend,2015-05-31 00:00:00,2099-12-31 00:00:00,2099-12-31 00:00:00,2099-12-31 00:00:00,2099-12-31 00:00:00
reimbt_cat_desc_fr,Radio-isotopes - Phospore 32 (P32),Spécialité pharmaceutique forfaitarisée autre ...,Produits radio-pharmaceutiques à base d’ isoto...,Produits radio-pharmaceutiques à base d’ isoto...,Spécialité pharmaceutique - médicaments à 90% ...
reimbt_cat_desc_nl,Radio-isotopes - Fosfor 32,Geforfaitariseerd geneesmiddel ander acuut zkh...,Radiofarmaceutische producten op basis van iso...,Radiofarmaceutische producten op basis van iso...,Farmaceutische specialiteit - geneesmiddelen a...
patient_cat,HOSP,HOSP,AMB,AMB,AMB
reimbt_crit_long,A,90-Fa,Ri-D7,Ri-D9,90-C
reimbt_crit_short,A,A,B,B,C
reimbt_cat_acute_yn,1,1,1,1,1
reimbt_cat_chron_yn,1,0,1,1,1


In [18]:
reimburse_cat["reimbt_crit_long"].unique()

array(['nd', 'A', 'B', 'Cat 1', 'Cat 2 (A)', 'Cat 3', 'Cat 4',
       'Cat 5 (D)', 'C', 'Cs', 'Cx', 'Cxg', 'D', 'Csg', 'Ag', 'Bg', 'Cg',
       'Forf Ant', 'Nutri Par', 'Br', 'Ar', 'Cr', 'Csr', 'Cxr',
       'Forf Adm', 'Forf BH', nan, 'V08', 'Fa', 'Fb', 'Forf 1-3',
       'Forf 4-', 'Ri-D11', 'Ri-T1', 'Ri-T2', 'Ri-T3', 'Ri-D5', 'Ri-D7',
       'Ri-D2', 'Ri-D9', 'Ri-D6', 'Ri-D10', 'Ri-D3', 'Ri-D1', 'Ri-D8',
       'Ri-T4', 'Ri-D4', 'Forf PET', '90-A', '90-B', '90-Fa', '90-Fb',
       'Ri-T5', 'Ri-T6', 'Ri-T7', 'Ri-T8', '90-C', '90-Cs', '90-Cx'],
      dtype=object)

In [22]:
# what do we do with nan? removed for now
# '90-C' is it 'other' or C ?
reimburse_cat_dict = {'nd':'other', 'A':'A', 'B':'B', 'Cat 1':'other', 'Cat 2 (A)':'other', 'Cat 3':'other', 'Cat 4':'other',
       'Cat 5 (D)':'other', 'C':'C', 'Cs':'C', 'Cx':'C', 'Cxg':'C', 'D':'D', 'Csg':'C', 'Ag':'A', 'Bg':'B', 'Cg':'C',
       'Forf Ant':'other', 'Nutri Par':'other', 'Br':'B', 'Ar':'A', 'Cr':'C', 'Csr':'C', 'Cxr':'C',
       'Forf Adm':'other', 'Forf BH':'other', 'V08':'other', 'Fa':'other', 'Fb':'other', 'Forf 1-3':'other',
       'Forf 4-':'other', 'Ri-D11':'other', 'Ri-T1':'other', 'Ri-T2':'other', 'Ri-T3':'other', 'Ri-D5':'other', 'Ri-D7':'other',
       'Ri-D2':'other', 'Ri-D9':'other', 'Ri-D6':'other', 'Ri-D10':'other', 'Ri-D3':'other', 'Ri-D1':'other', 'Ri-D8':'other',
       'Ri-T4':'other', 'Ri-D4':'other', 'Forf PET':'other', '90-A':'other', '90-B':'other', '90-Fa':'other', '90-Fb':'other',
       'Ri-T5':'other', 'Ri-T6':'other', 'Ri-T7':'other', 'Ri-T8':'other', '90-C':'other', '90-Cs':'other', '90-Cx':'other'}

In [27]:
reimburse_cat_dict

{'90-A': 'other',
 '90-B': 'other',
 '90-C': 'other',
 '90-Cs': 'other',
 '90-Cx': 'other',
 '90-Fa': 'other',
 '90-Fb': 'other',
 'A': 'A',
 'Ag': 'A',
 'Ar': 'A',
 'B': 'B',
 'Bg': 'B',
 'Br': 'B',
 'C': 'C',
 'Cat 1': 'other',
 'Cat 2 (A)': 'other',
 'Cat 3': 'other',
 'Cat 4': 'other',
 'Cat 5 (D)': 'other',
 'Cg': 'C',
 'Cr': 'C',
 'Cs': 'C',
 'Csg': 'C',
 'Csr': 'C',
 'Cx': 'C',
 'Cxg': 'C',
 'Cxr': 'C',
 'D': 'D',
 'Fa': 'other',
 'Fb': 'other',
 'Forf 1-3': 'other',
 'Forf 4-': 'other',
 'Forf Adm': 'other',
 'Forf Ant': 'other',
 'Forf BH': 'other',
 'Forf PET': 'other',
 'Nutri Par': 'other',
 'Ri-D1': 'other',
 'Ri-D10': 'other',
 'Ri-D11': 'other',
 'Ri-D2': 'other',
 'Ri-D3': 'other',
 'Ri-D4': 'other',
 'Ri-D5': 'other',
 'Ri-D6': 'other',
 'Ri-D7': 'other',
 'Ri-D8': 'other',
 'Ri-D9': 'other',
 'Ri-T1': 'other',
 'Ri-T2': 'other',
 'Ri-T3': 'other',
 'Ri-T4': 'other',
 'Ri-T5': 'other',
 'Ri-T6': 'other',
 'Ri-T7': 'other',
 'Ri-T8': 'other',
 'V08': 'other',
 'nd': 'ot

In [30]:
reimburse_cat["reimbt_crit_long"].map(reimburse_cat_dict).isnull().count()

708

In [37]:
df = df.merge(pharma_ref, how="inner", left_on="drug_code", right_on="drug_code").reset_index()

In [38]:
df

,index,Province,Type,hosp_serv_id,reimbt_cat_id,drug_code,realization_date,quantity,amount_reimb,amount_not_reimb,...,code_atc,code_atc_5,code_atc_4,code_atc_3,code_atc_1,DDD,nombre_prises,orphan_flag,chapter_IV_bis_flag,link_same_tablet
0,0,Anvers,Général,990,698014,0,20014,1,218.10,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
1,1,Anvers,Général,990,698051,0,20014,3,5816.01,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
2,2,Anvers,Général,990,698051,0,20014,1,51.83,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
3,3,Anvers,Général,990,698051,0,20021,2,535.21,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
4,4,Anvers,Général,990,698051,0,20021,1,267.61,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
5,5,Anvers,Général,720,698051,0,20022,1,5184.17,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
6,6,Anvers,Général,720,698051,0,20022,1,5184.17,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
7,7,Anvers,Général,990,698051,0,20022,7,6789.83,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
8,8,Anvers,Général,990,698051,0,20023,4,1070.44,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995
9,9,Anvers,Général,220,698062,0,20012,1,676.70,0.00,...,X00XX00,X00XX,X00X,X00,X,0.0,0,0,0,-11995


In [39]:
! pwd

/home/ubuntu/venv/dataforbetterhealth/DataforbetterHealth/code


In [40]:
df.to_csv("../data/dataset/DB_V3.csv", index=False, encoding="latin-1")